In [1]:
import affine as af

In [19]:
bt.__version__

'9.8.2'

In [20]:
import bittensor as bt
subtensor = bt.async_subtensor(network='finney')
await subtensor.initialize()
await subtensor.metagraph( 5)


metagraph(netuid:5, n:256, block:6010560, network:finney)

In [13]:
await subtensor.metagraph( 5)

ValueError: Runtime API Call 'NeuronInfoRuntimeApi.get_neurons_lite' not found in registry

In [ ]:
import bittensor as bt
subtensor = bt.async_subtensor(network='subvortex')
await subtensor.initialize()
   

In [27]:
from collections import defaultdict

def compute_ranks(submissions, env_scores):
    """
    submissions: list of keys/IDs
    env_scores: dict e→dict s→score (higher is better but we only care about ordering)
    Returns: ranks[e][s] where rank 1 is top performer in env e
    """
    ranks = {}
    for e, scores in env_scores.items():
        # sort submissions descending by score, break ties arbitrarily
        sorted_s = sorted(submissions, key=lambda s: scores[s], reverse=True)
        ranks[e] = { s: i+1 for i, s in enumerate(sorted_s) }
    return ranks

def dominance_counts(submissions, ranks):
    """
    For each submission s, count how many t≠s such that
      for ALL envs e: ranks[e][s] < ranks[e][t]
    Returns: dict s→count
    """
    envs = list(ranks.keys())
    count = defaultdict(int)
    for s in submissions:
        for t in submissions:
            if s == t: continue
            # check strict beating in every env
            if all(ranks[e][s] < ranks[e][t] for e in envs):
                count[s] += 1
    return count

def find_best(submissions, env_scores):
    """
    Returns the submission with the largest dominance count.
    """
    ranks = compute_ranks(submissions, env_scores)
    counts = dominance_counts(submissions, ranks)
    # pick s with max count (ties broken arbitrarily or by secondary key)
    return max(submissions, key=lambda s: counts[s])

# Example usage:
subs = ["a","b","c","d", "f", "g"]
scores = {
    "SWE": {"a":.9,"b":.8,"c":.85,"d":1.0,"f":1.0,"g":1.0},
    "POLY": {"a":.7,"b":.75,"c":.65,"d":.3,"f":0.1,"g":1.0},
    "SYNTH": {"a":.7,"b":.75,"c":.65,"d":.3,"f":1.0,"g":1.0},
}
best = find_best(subs, scores)
print("Best submission:", best)

Best submission: g


In [30]:
from collections import defaultdict

def compute_ranks(submissions, env_scores):
    ranks = {}
    for e, scores in env_scores.items():
        sorted_s = sorted(submissions, key=lambda s: scores[s], reverse=True)
        ranks[e] = { s: i+1 for i, s in enumerate(sorted_s) }
    return ranks

def dominance_counts(submissions, ranks):
    envs = list(ranks.keys())
    count = defaultdict(int)
    for s in submissions:
        for t in submissions:
            if s == t:
                continue
            if all(ranks[e][s] < ranks[e][t] for e in envs):
                count[s] += 1
    return count

def find_best(submissions, env_scores):
    ranks = compute_ranks(submissions, env_scores)
    counts = dominance_counts(submissions, ranks)
    # return both winner and the full count dict
    winner = max(submissions, key=lambda s: counts[s])
    return winner, counts

def simulate(decoy_count=50, attack_boost=0.0):
    # Honest field
    base_subs = ["alice", "bob", "carol"]
    attacker = "eve"
    subs = base_subs + [attacker]

    # Base scores (higher = better)
    base_scores = {
        "SWE":   {"alice": .90, "bob": .85,  "carol": .88,  "eve": .89 + attack_boost},
        "POLY":  {"alice": .80, "bob": .82,  "carol": .83,  "eve": .81 + attack_boost},
        "SYNTH": {"alice": .85, "bob": .87,  "carol": .86,  "eve": .84 + attack_boost},
    }

    # 1) Before decoys
    best_before, counts_before = find_best(subs, base_scores)
    print("Without decoys:")
    print("  Dominance counts:", dict(counts_before))
    print("  Winner:", best_before)

    # 2) Inject decoys
    decoys = [f"decoy{i}" for i in range(1, decoy_count + 1)]
    subs_all = subs + decoys
    # Assign very low scores to each decoy
    for env in base_scores:
        for d in decoys:
            base_scores[env][d] = 0.01

    best_after, counts_after = find_best(subs_all, base_scores)
    print(f"\nWith {decoy_count} decoys:")
    print("  Dominance counts (excerpt):",
          {s: counts_after[s] for s in subs + decoys[:3]})
    print("  Winner:", best_after)

    # 3) Correct success condition
    if best_after == attacker:
        print("\n-- Attack successful! Attacker became the winner. --")
    else:
        print("\n-- Attack failed. Attacker did not win. --")

print("=== Case 1: attacker with NO boost ===")
simulate(decoy_count=100, attack_boost=0.0)

print("\n=== Case 2: attacker with small boost (0.02) ===")
simulate(decoy_count=100, attack_boost=0.02)


=== Case 1: attacker with NO boost ===
Without decoys:
  Dominance counts: {'alice': 0, 'bob': 0, 'carol': 0, 'eve': 0}
  Winner: alice

With 100 decoys:
  Dominance counts (excerpt): {'alice': 100, 'bob': 100, 'carol': 100, 'eve': 100, 'decoy1': 99, 'decoy2': 98, 'decoy3': 97}
  Winner: alice

-- Attack failed. Attacker did not win. --

=== Case 2: attacker with small boost (0.02) ===
Without decoys:
  Dominance counts: {'eve': 1, 'alice': 0, 'bob': 0, 'carol': 0}
  Winner: eve

With 100 decoys:
  Dominance counts (excerpt): {'alice': 100, 'bob': 100, 'carol': 100, 'eve': 101, 'decoy1': 99, 'decoy2': 98, 'decoy3': 97}
  Winner: eve

-- Attack successful! Attacker became the winner. --
